In [27]:
import pandas as pd
# To save model weights and features
import pickle
# models
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from model_config import *

# Retrieve all words in all files for a large corpus

In [2]:
# read_direction_analysis function from direction_and_file_handleing file

all_direction_words = read_direction_analysis(preprocessed_direction_path_tweets, "")
# all_direction_words = all_direction_words[:len(all_direction_words) - 3000000]
print("The total words of January Month are: ", len(all_direction_words))

The total words of January Month are:  304406


## Read and Shuffle Model Data

In [3]:
df_file = pd.read_csv(analysis_direction_path_tweets + 'train_test_data/last_classifed_tweets_to_model_on_over_15000_tweets.csv', lineterminator='\n', error_bad_lines=False)
df_file.dropna(inplace=True)
print(len(df_file))
print("The number of class 1 which represent this tweet talks about coronavirus: ", len(df_file[df_file['class'] == 1]))
print("The number of class 0 which represent this tweet not talks about coronavirus: ", len(df_file[df_file['class'] == 0]))
df_file.head()

15075
The number of class 1 which represent this tweet talks about coronavirus:  7267
The number of class 0 which represent this tweet not talks about coronavirus:  7808


,class,tweet_text
0,1,طب مسلمين الصين اللي بيموتو الفيرس ذنب برده ا...
1,1,علي ما اذكر معظم الفيروسات ما عدا الايبولا ك...
2,1,اميركا السبب لضرب الاقتصاد الصيني وتركيع الصين...
3,1,قلنا نقاطع الصين الناس قالت مش ممكن الان مق...
4,1,نصحيتين اقدمها لك 1 لا تتروش في البرد 2 اح...


In [4]:
start_train_set =pd.read_csv(analysis_direction_path_tweets + 'train_test_data/train_set.csv',  lineterminator='\n', error_bad_lines=False)
start_train_set.dropna(inplace=True)
start_test_set  = pd.read_csv(analysis_direction_path_tweets + 'train_test_data/test_set.csv',  lineterminator='\n', error_bad_lines=False)
start_test_set.dropna(inplace=True)

In [5]:
# look how the train set is shuffle
start_train_set.dropna()
print(start_train_set.info())
start_train_set.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12060 entries, 0 to 12059
Data columns (total 2 columns):
class         12060 non-null int64
tweet_text    12060 non-null object
dtypes: int64(1), object(1)
memory usage: 282.7+ KB
None


,class,tweet_text
0,0,استقدمو الصين وزراء ربعنا ماههم كفو
1,0,اكثر ما نستطيع فعله مقاطعه الصين باكبر قدر مم...
2,0,سمعنا_صوتك_ياهادي الحمله حملت سفير الصين علي ...
3,0,اوزيل حتي باشد فتراته سوءا كان يتحفنا بتمريرات...
4,0,لك بان الامريكان اقويا فقد كذب عزيزي قوه الامر...


In [6]:
# look how the test set is shuffle 
start_test_set.dropna()
print(start_test_set.info())
start_test_set.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3014 entries, 0 to 3014
Data columns (total 2 columns):
class         3014 non-null int64
tweet_text    3014 non-null object
dtypes: int64(1), object(1)
memory usage: 70.6+ KB
None


,class,tweet_text
0,1,ربنا احفظك واحميك واغطي واردك سالم غانم وكل اخ...
1,0,الرياض\n الصين تستبدل الالعاب الناريه والليز...
2,1,لو سمعحتوا فعلا الفايرس كورونا ياثر علي المنتج...
3,1,عاجل بكين تعلن حاله وفاه في العاصمه فيروس كو...
4,0,المطلوب التركيز علي ادامه زخم دعوات الانسحاب ا...


# Features Engineering

In [7]:
tweets_train, target_train, tweets_test, target_test = split_train_test_set(start_train_set, start_test_set)

Number of Training Tweets is:  12060
Number of Tagets class in Training Tweets is:  12060
Number of Testing Tweets is:  3014
Number of Tagets class in Testing Tweets is:  3014


In [8]:
# tfidf_vectorizer function from features_engineering file in config_files direction
file_path = 'models_weights/tf_idf/tf_idf_vectorizer_50000_fetures.pickle'
tfidf_vectorizer = load_tfidf_vectorizer_for_supervised_models(file_path)

In [9]:
training_data = tfidf_vectorizer.transform(tweets_train)
training_data_array = training_data.toarray()
print("Our new vectorized data: " + str(training_data_array.shape))
print("The first 2 review after transform: \n", training_data_array[:2])

Our new vectorized data: (12060, 50000)
The first 2 review after transform: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
testing_data = tfidf_vectorizer.transform(tweets_test)
testing_data_array = testing_data.toarray()
print("Our new vectorized data: " + str(testing_data_array.shape))
print("The first 2 review after transform: \n", testing_data_array[:2])

Our new vectorized data: (3014, 50000)
The first 2 review after transform: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
training_data_array = training_data_array[:9000]


In [11]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(training_data_array)

KMeans(n_clusters=2)

In [18]:
predict_k = kmeans.predict(training_data_array)
real_class = target_train[:9000]

In [19]:
# clf_LogisticRegression1 = pickle.load(open("models_weights/logistic_regression/clf_LogisticRegression1_93tr_88ts_50000fetures.sav", "rb"))
# tfidf_vectorizer = pickle.load(open("models_weights/tf_idf/tf_idf_vectorizer_70000_fetures.pickle", "rb"))


In [20]:
# x_kmean[:3]

In [21]:
# data_transform = tfidf_vectorizer.transform(x_kmean)
# data_transform = data_transform.toarray()
# predict_lo = clf_LogisticRegression1.predict(data_transform)

In [22]:
np.mean(predict_k == real_class)

0.6882222222222222

In [24]:
predict_k = kmeans.predict(testing_data_array)
# real_class = target_train[:9000]
total = sum(predict_k == target_test)
total

2029

In [25]:
total/len(target_test)

0.6731917717319177

In [26]:
pickle.dump(kmeans, open('models_weights/kmeans/kmeans_68tr_67ts.sav', 'wb'))
